In [3]:
import requests
import json
from base64 import b64encode
import os
from dotenv import load_dotenv

In [6]:
# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variables
api_token_string = os.getenv('API_KEY')

# Check if the API key was loaded correctly
if api_token_string is None:
    raise ValueError("API_KEY environment variable not set")

api_token_string = api_token_string + ':api_token'    

In [9]:
TOGGL_API_URL = 'https://api.track.toggl.com/api/v9/me'

headers = {
            'content-type': 'application/json',
            'Authorization' : 'Basic %s' %  b64encode(api_token_string.encode('utf-8')).decode("ascii")
          }

In [ ]:
response = requests.get(TOGGL_API_URL + '/time_entries?since=1719957600', headers=headers)

# Ensure the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Pretty-print the JSON response
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print(f"Request failed with status code {response.status_code}")